In [38]:
import torch
from torch import nn
import sys

In [39]:
config = [
    (32, 3, 1),
    (64, 3, 2),
    ["B", 1],
    (128, 3, 2),
    ["B", 2],
    (256, 3, 2),
    ["B", 8],
    (512, 3, 2),
    ["B", 8],
    (1024, 3, 2),
    ["B", 4],
    (512, 1, 1),
    (1024, 3, 1),
    "S",
    (256, 1, 1),
    "U",
    (256, 1, 1),
    (512, 3, 1),
    "S",
    (128, 1, 1),
    "U",
    (128, 1, 1),
    (256, 3, 1),
    "S",
]

In [40]:
class CNNBlock(nn.Module):
    def __init__(self, in_channels, out_channels, bn_act=True, **kwargs):
        super().__init__()
        self.conv = nn.Conv2d(in_channels, out_channels, bias=not bn_act, **kwargs)
        self.bn = nn.BatchNorm2d(out_channels)
        self.leaky = nn.LeakyReLU(0.1)
        self.use_bn_act = bn_act
    
    def forward(self, x):
        if self.use_bn_act:
            x = self.leaky(self.bn(self.conv(x)))
        else:
            x = self.conv(x)
        return x

class Residualblock(nn.Module):
    def __init__(self, in_channels, use_residual=True, num_repeats=1):
        super().__init__()
        self.layers = nn.ModuleList()
        
        for repeat in range(num_repeats):
            self.layers += [
            nn.Sequential(
                CNNBlock(in_channels, in_channels//2, kernel_size=1),
                CNNBlock(in_channels//2, in_channels, kernel_size=3, padding=1)
                )
            ]
            
    def forward(self, x):
        for layer in self.layers:
            if self.use_residual:
                x = layer(x) + x
            else:
                x = layer()
        return x
    
class ScalePrediction(nn.Module):
    def __init__(self, in_channels, num_classes):
        super().__init__()
        self.pred = nn.Sequential(
            CNNBlock(in_channels, 2 * in_channels, kernel_size=3, padding=1),
            CNNBlock(2 * in_channels, (num_classes + 5) * 3, bn_act=False, kernel_size=1)
        )
        self.num_classes = num_classes
        
    def forward(self, x):
        x = (self.pred(x)
        .reshape(x[0], 3, (self.num_classes+5), x[2], x[3])
        .permute(0, 1, 3, 4, 2)
            )
            
        return x
     
    

In [41]:
class YOLOV3(nn.Module):
    def __init__(self, in_channels=3, num_classes=20):
        super().__init__()
        self.layers = self._create_conv_layers() 
        self.in_channels = in_channels
        self.num_classes = num_classes
        
    def forward(self, x):
        outputs = []
        route_connections = []
        
        for layer in self.layers:
            if isinstance(layer, ScalePrediction):
                outputs.append(layer(x))
                continue
                
            x = layer(x)
            
            if isinstance(layer, Residualblock) and layer.num_repeats == 8:
                route_connections.append(x)
                
            if isinstance(layer, nn.Upsample):
                x = torch.cat([x, route_connections[-1]], dim=1)
                route_connections.pop()
                
        return outputs       
            
    
    def _create_conv_layers(self):
        print(self.in_channels)
        sys.exit()
        layers = nn.ModuleList()
        
        for module in config:
            if isinstance(module, tuple):
                out_channels, kernel_size, stride = module
                layers += [
                    CNNBlock(self.in_channels, out_channels, kernel_size=kernel_size, stride=stride,
                             padding=1 if kernel_size==3 else 0)
                ]
                
                self.in_channels = out_channels
                
            elif isinstance(module, list):
                num_repeats = module[1]
                layers += [
                    Residualblock(self.in_channels, num_repeats=num_repeats)
                ]
                
            elif isinstance(module, str):
                if module == "S":
                    layers += [
                        Residualblock(self.in_channels, use_residual=False),
                        CNNBlock(self.in_channels, self.in_channels//2, kernel_size=1),
                        ScalePrediction(self.in_channels//2, num_classes=self.num_classes)
                    ]
                    
                    self.in_channels = self.in_channels//2
                
                elif module == "B":
                    layers.append(nn.Upsample(scale_factor=2))
                    self.in_channels = self.in_channels * 3
                    
            
        return layers

In [43]:
if __name__ == "__main__":
    num_classes = 20
    IMAGE_SIZE = 416
    model = YOLOV3()
    x = torch.randn((2, 3, IMAGE_SIZE, IMAGE_SIZE))
    out = model(x)
    assert model(x)[0].shape == (2, 3, IMAGE_SIZE//32, IMAGE_SIZE//32, num_classes + 5)
    assert model(x)[1].shape == (2, 3, IMAGE_SIZE//16, IMAGE_SIZE//16, num_classes + 5)
    assert model(x)[2].shape == (2, 3, IMAGE_SIZE//8, IMAGE_SIZE//8, num_classes + 5)
    print("Success!")

AttributeError: 'YOLOV3' object has no attribute 'in_channels'